In [96]:
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
sentences = open('data/tinyshakespeare.txt').read()

In [12]:
chars = sorted(set(sentences))

In [21]:
itoc = {i:c for i,c in enumerate(chars)}
ctoi = {c:i for i,c in itoc.items()}

In [23]:
len(sentences)

1115394

In [24]:
len(chars)

65

In [25]:
N = torch.zeros((65,65), dtype=torch.int32)

In [26]:
for i in range(len(sentences) - 1):
    ix1 = ctoi[sentences[i]]
    ix2 = ctoi[sentences[i+1]]
    N[ix1,ix2] += 1

In [31]:
P = (N+1).float()
P /= P.sum(1, keepdim=True)

In [52]:
for i in range(1):
    out = []
    ix = 0
    for _ in range(50):
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True).item()
        out.append(itoc[ix])
    print(''.join(out))


A oe d Homano le.
Catare, nge.
TES loteth te anto


Neural Network based approach

In [53]:
# build dataset
xs, ys = [], []
for ch1, ch2 in zip(sentences, sentences[1:]):
    ix1 = ctoi[ch1]
    ix2 = ctoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [56]:
xs.shape, ys.shape

(torch.Size([1115393]), torch.Size([1115393]))

In [60]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [59]:
class MLP(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(len(chars), hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, len(chars))
        )

    def forward(self, x):
        return self.net(x)

In [61]:
model = MLP(100)

In [76]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [98]:
for n in range(10000):
    batch_indices = torch.randint(0, xs.shape[0] - 1, (64,))
    inp = F.one_hot(xs[batch_indices], num_classes=65).float()
    tgt = F.one_hot(ys[batch_indices], num_classes=65).float()
    logits = model(inp)
    loss = F.cross_entropy(logits, tgt)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

In [95]:
for i in range(1):
    out = []
    ix = torch.randint(0, len(chars), (1,))

    for _ in range(50):
        xenc = F.one_hot(torch.tensor([ix]), num_classes=65).float()
        logits = model(xenc)
        probs = F.softmax(logits, dim=-1)
        ix = torch.multinomial(probs, num_samples=1, replacement=True).item()
        out.append(itoc[ix])
    print(''.join(out))

GenceeshecirounE:
To;
For f, t!
LN-udyIWha arWIKoP
